## Predicting salaries using Logistic Regression

#### Load in the the data of scraped salaries

In [1]:
## YOUR CODE HERE
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn as sns

In [2]:
data = pd.read_csv('../assets/indeed-scraped-job-postings.csv')
data

,city,company,salary,summary,title,parsed_salary
0,Denver,Department Of The Interior,"$76,341 - $99,243 a year","Would you like to join the more than 10,000 sc...","Statistician, GS-1350-12 (DEU-PERM-DS)",87792.0
1,Denver,Department Of The Interior,"$71,012 - $99,243 a year",Investigate potential uses of geospatial data ...,Interdisciplinary Cartographer/Geographer - GS...,85127.5
2,Denver,Mental Health Center of Denver,"$70,000 - $80,000 a year",Advise the Data Developer with regard to creat...,Financial Data Scientist,75000.0
3,Denver,Denver Public Schools,"$62,712 - $75,255 a year",Portal managers on student outcome data report...,SENIOR RESEARCH ANALYST,68983.5
4,Denver,University of Colorado,"$25,000 - $29,000 a year",Experience entering and manipulating data in a...,Animal Care I,27000.0
5,Denver,Denver Health,"$93,645 a year",Functions largely independently but reports to...,Assistant Research Scientist - RADARS,93645.0
6,Denver,Denver Health,"$66,654 a year","Assist research director, assistant research d...",Statistical Research Specialist,66654.0
7,Denver,Denver Health,"$66,654 a year","Assist research director, assistant research d...",Statistical Research Scientist - RADARS,66654.0
8,Denver,University of Colorado,"$65,000 - $90,000 a year",And effective working relationships with physi...,Regulatory & Compliance Manager,77500.0
9,Denver,"Computer Enterprises, Inc.","$130,000 a year",We are looking for a Software Engineer/Data Sc...,Software Engineer/Data Scientist,130000.0


In [3]:
parsed_salary_null = pd.isnull(data['parsed_salary'])
data_science_salary_data_missing = data[parsed_salary_null == True]
Salary_data_missing_rows = data_science_salary_data_missing.index.values
Salary_data_missing_rows

array([ 41, 123, 124, 189, 310, 367, 389], dtype=int64)

In [4]:
Parsed_salary_missing_data = [(100/8)*2000,(120/8)*2000,
                              (120/8)*2000,(775/8)*2000,
                              (150/8)*2000,(82.5/8)*2000,(135/8)*2000]
Missing_salary = zip(Salary_data_missing_rows, Parsed_salary_missing_data)
for i,j in Missing_salary:
    data.at[i,'parsed_salary'] = j
data.head()

,city,company,salary,summary,title,parsed_salary
0,Denver,Department Of The Interior,"$76,341 - $99,243 a year","Would you like to join the more than 10,000 sc...","Statistician, GS-1350-12 (DEU-PERM-DS)",87792.0
1,Denver,Department Of The Interior,"$71,012 - $99,243 a year",Investigate potential uses of geospatial data ...,Interdisciplinary Cartographer/Geographer - GS...,85127.5
2,Denver,Mental Health Center of Denver,"$70,000 - $80,000 a year",Advise the Data Developer with regard to creat...,Financial Data Scientist,75000.0
3,Denver,Denver Public Schools,"$62,712 - $75,255 a year",Portal managers on student outcome data report...,SENIOR RESEARCH ANALYST,68983.5
4,Denver,University of Colorado,"$25,000 - $29,000 a year",Experience entering and manipulating data in a...,Animal Care I,27000.0


In [5]:
data_scientist = ['statistical', 'statistician', 'machine learning', 'scientist', 'science', 'big data', 'research', 'researcher',
'deep learning', 'nlp', 'natural language processing', 'predictive ',
'hadoop', 'modeling', 'modeler', 'time series', 'postdoctoral', 'phd']


In [6]:
data_analyst = ['analyst', 'analysis', 'business intelligence',
                'analytics', 'intelligence', 'data administrator', 'sql']

In [7]:
engineer = ['developer', 'C++', 'java', 'software', 'engineer', 'python', 'devops', 
'product development', '.js', 'front end', 'applications', 'application']

In [8]:
solution_architect = ['architect', 'system', 'network', 'solution']

In [9]:
field = [data_scientist, data_analyst, engineer, solution_architect]

In [10]:
data['title_lowercase'] = data['title']
data['title_lowercase'] = data.title_lowercase.str.lower()
data['title_keywords'] = 0
data.head(1)

,city,company,salary,summary,title,parsed_salary,title_lowercase,title_keywords
0,Denver,Department Of The Interior,"$76,341 - $99,243 a year","Would you like to join the more than 10,000 sc...","Statistician, GS-1350-12 (DEU-PERM-DS)",87792.0,"statistician, gs-1350-12 (deu-perm-ds)",0


In [11]:
data.title_lowercase

0                 statistician, gs-1350-12 (deu-perm-ds)
1      interdisciplinary cartographer/geographer - gs...
2                               financial data scientist
3                                senior research analyst
4                                          animal care i
5                  assistant research scientist - radars
6                        statistical research specialist
7                statistical research scientist - radars
8                        regulatory & compliance manager
9                       software engineer/data scientist
10     global marketing manager - ingredient and scho...
11             geologist gs-1350-07/09 (deu - term - jg)
12        professional research assistant - data manager
13     professional research assistant - tissue bank ...
14       platform developer/geospatial big data (python)
15     professional research assistant - clinical res...
16                                       devops engineer
17                  data scient

In [24]:
for i, title in enumerate(data.title_lowercase):
    a = title.split()
    for b in a:
        if b in data_scientist:
            data.title_keywords[i] = 'data scientist'
            break
        if b in data_analyst:
            data.title_keywords[i] = 'data analyst'
            break
        if b in engineer:
            data.title_keywords[i] = 'engineer'
            break 
        if b in solution_architect:
            data.title_keywords[i] = 'solution architect'
        else:
            data.title_keywords[i] = 'other'


C:\Users\ali\Anaconda2\lib\site-packages\ipykernel\__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ali\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ali\Anaconda2\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ali\Anaconda2\lib\site-packages\ipykernel\__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame


In [25]:
data

,city,company,salary,summary,title,parsed_salary,title_lowercase,title_keywords
0,Denver,Department Of The Interior,"$76,341 - $99,243 a year","Would you like to join the more than 10,000 sc...","Statistician, GS-1350-12 (DEU-PERM-DS)",87792.0,"statistician, gs-1350-12 (deu-perm-ds)",other
1,Denver,Department Of The Interior,"$71,012 - $99,243 a year",Investigate potential uses of geospatial data ...,Interdisciplinary Cartographer/Geographer - GS...,85127.5,interdisciplinary cartographer/geographer - gs...,other
2,Denver,Mental Health Center of Denver,"$70,000 - $80,000 a year",Advise the Data Developer with regard to creat...,Financial Data Scientist,75000.0,financial data scientist,data scientist
3,Denver,Denver Public Schools,"$62,712 - $75,255 a year",Portal managers on student outcome data report...,SENIOR RESEARCH ANALYST,68983.5,senior research analyst,data scientist
4,Denver,University of Colorado,"$25,000 - $29,000 a year",Experience entering and manipulating data in a...,Animal Care I,27000.0,animal care i,other
5,Denver,Denver Health,"$93,645 a year",Functions largely independently but reports to...,Assistant Research Scientist - RADARS,93645.0,assistant research scientist - radars,data scientist
6,Denver,Denver Health,"$66,654 a year","Assist research director, assistant research d...",Statistical Research Specialist,66654.0,statistical research specialist,data scientist
7,Denver,Denver Health,"$66,654 a year","Assist research director, assistant research d...",Statistical Research Scientist - RADARS,66654.0,statistical research scientist - radars,data scientist
8,Denver,University of Colorado,"$65,000 - $90,000 a year",And effective working relationships with physi...,Regulatory & Compliance Manager,77500.0,regulatory & compliance manager,other
9,Denver,"Computer Enterprises, Inc.","$130,000 a year",We are looking for a Software Engineer/Data Sc...,Software Engineer/Data Scientist,130000.0,software engineer/data scientist,engineer


In [26]:
data.title_keywords.unique()

array(['other', 'data scientist', 'engineer', 'solution architect',
       'data analyst'], dtype=object)

In [12]:

for i, title in enumerate(data.title_lowercase):
    for ds in data_scientist:
        if ds in title:
            data.title_keywords[i] = 'data scientist'
        else:
            for an in data_analyst:
                if an in title:
                    data.title_keywords[i] = 'data analyst'
#                 else:
#                     for eng in engineer:
#                         if eng in title:
#                             data.title_keywords[i] = 'engineer'
#                         else:
#                             for sa in solution_architect:
#                                 if sa in title:
#                                     data.title_keywords[i] = 'engineer'
#                                 else:
#                                     data.title_keywords[i] = 'other'

C:\Users\ali\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ali\Anaconda2\lib\site-packages\pandas\core\indexing.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\ali\Anaconda2\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [13]:
data

,city,company,salary,summary,title,parsed_salary,title_lowercase,title_keywords
0,Denver,Department Of The Interior,"$76,341 - $99,243 a year","Would you like to join the more than 10,000 sc...","Statistician, GS-1350-12 (DEU-PERM-DS)",87792.0,"statistician, gs-1350-12 (deu-perm-ds)",data scientist
1,Denver,Department Of The Interior,"$71,012 - $99,243 a year",Investigate potential uses of geospatial data ...,Interdisciplinary Cartographer/Geographer - GS...,85127.5,interdisciplinary cartographer/geographer - gs...,0
2,Denver,Mental Health Center of Denver,"$70,000 - $80,000 a year",Advise the Data Developer with regard to creat...,Financial Data Scientist,75000.0,financial data scientist,data scientist
3,Denver,Denver Public Schools,"$62,712 - $75,255 a year",Portal managers on student outcome data report...,SENIOR RESEARCH ANALYST,68983.5,senior research analyst,data analyst
4,Denver,University of Colorado,"$25,000 - $29,000 a year",Experience entering and manipulating data in a...,Animal Care I,27000.0,animal care i,0
5,Denver,Denver Health,"$93,645 a year",Functions largely independently but reports to...,Assistant Research Scientist - RADARS,93645.0,assistant research scientist - radars,data scientist
6,Denver,Denver Health,"$66,654 a year","Assist research director, assistant research d...",Statistical Research Specialist,66654.0,statistical research specialist,data scientist
7,Denver,Denver Health,"$66,654 a year","Assist research director, assistant research d...",Statistical Research Scientist - RADARS,66654.0,statistical research scientist - radars,data scientist
8,Denver,University of Colorado,"$65,000 - $90,000 a year",And effective working relationships with physi...,Regulatory & Compliance Manager,77500.0,regulatory & compliance manager,0
9,Denver,"Computer Enterprises, Inc.","$130,000 a year",We are looking for a Software Engineer/Data Sc...,Software Engineer/Data Scientist,130000.0,software engineer/data scientist,data scientist


In [15]:
data.title_keywords.unique()

array(['data scientist', 0L, 'data analyst'], dtype=object)